In [1]:
# Standard modules
from dataclasses import dataclass
from logging import getLogger, StreamHandler, DEBUG, config
import os

# Third party modules
import gffutils
import pybedtools
import pandas as pd
import numpy as np
from pathlib2 import Path
import pysam
from tqdm import tqdm
import yaml

# Local modules
from libs.args import parser_setting
from libs.utils import load_config, OutputSettings
from libs.preprocess import PreProcessExomeSummary
from libs.modesamples import ModeSamples
from libs.annolibs.anno import Anno
from libs.annolibs.genebased import GeneBasedAnno
from libs.filter.maffilter import MafFilter
from libs.filter.typefilter import TypeFilter
from libs.filter.gtfilter import GtFilter
from libs.filter.counter import counter

# Settings
tqdm.pandas()

#----- STEP 0. Logging settings
# parent_directory = os.path.dirname(os.path.dirname(__file__))
parent_directory = os.path.dirname(os.path.abspath('../__file__'))
config_path: str = os.path.join(parent_directory, 'config/logging.yaml')
with open(config_path, 'r') as f:
    config.dictConfig(yaml.safe_load(f))
logger = getLogger(__name__)

# http://www.openbioinformatics.org/annovar/download/0wgxR2rIVP/annovar.latest.tar.gz

INFO: Pandarallel will run on 4 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [2]:
#-----   STEP 1. Argument settings
args = parser_setting()

#-----   STEP 2. Input file and Output settings
df = pd.read_table(args['input'], header=0, dtype=str)
configs: dict = load_config(args['config'])

#-----   STEP 3. Get Mode and Samples information
modesamples = ModeSamples(df=df, args=args)
mode_samples_info = modesamples.get_mode_samples_info()
logger.info(f"mode_samples_info: {mode_samples_info.mode}")

#-----   STEP 4. Output settings
output_settings = OutputSettings(
    args=args, mode_samples_info=mode_samples_info)
output_file_path = output_settings.get_saving_file_path()

#-----   STEP 5. Pre-processing
preprocessing = PreProcessExomeSummary(
    df=df, args=args, mode_samples_info=mode_samples_info)
df = preprocessing.all_pre_processing()

#-----   STEP 6. Annotation
genebasedanno = GeneBasedAnno(args['resources'])
df = genebasedanno.anno_hgmd(df=df)

#-----   STEP 7. Filtering
maffilter = MafFilter(
    df=df, mode_samples_info=mode_samples_info, config=configs)
df = maffilter.all_filtering()

typefilter = TypeFilter(df=df)
df = typefilter.exclude_hlamuc_and_exonicsyno()

gtfilter = GtFilter(
    df=df, mode_samples_info=mode_samples_info)
dfs = gtfilter.genotypeing_filter()


2024/01/16 02:39:45 [INFO   ] (libs.args) - Arguments are as follows:

     input: /Volumes/vol/work/Github/TestData/proband/IRUD_HRS/annovar/exome_summary.20230413_190252.txt
     output: None
     xhmm: /work/Github/TestData/proband/xhmm/data.segdup.strvar.haplo.deciph.omim.xcnv.gene.uniq
     phenotype: None
     mode: quad_unaffected
     samples: auto
     assembly: hg19
     config: /Volumes/vol/work/Github/playground/wesanno/config/config.toml
     resources: /Volumes/vol/work/Github/playground/wesanno/resources
     no_gnomad: False
     no_hgmd: False
     no_decipher: False
     no_ddg2p: False
     no_jarvis: False
     no_spliceai: False
     no_syno: None
     no_alphamissense: False
     no_revel: False
     no_trap: False
     excel_formating: True


2024/01/16 02:39:45 [INFO   ] (libs.preprocess) - Liftover to hg38 is started
2024/01/16 02:39:45 [INFO   ] (libs.preprocess) - It takes about 5 minutes.


2024/01/16 02:43:50 [INFO   ] (libs.preprocess) - Extract InHouse MAF


2024/01/16 02:43:58 [INFO   ] (libs.preprocess) - Extract MAF from each database cols
2024/01/16 02:43:58 [INFO   ] (libs.preprocess) - Extract genotypeing info
2024/01/16 02:43:58 [INFO   ] (libs.preprocess) - Split QC info
2024/01/16 02:43:58 [INFO   ] (libs.preprocess) - Replace "." to np.nan
2024/01/16 02:43:58 [INFO   ] (libs.preprocess) - Split ALT column
2024/01/16 02:44:00 [INFO   ] (libs.preprocess) - Drop unused columns


In [3]:

#-----   STEP 8. Output as an Excel 
def df_to_excel(dfs: dataclass, output_xlsx) -> None:
    sheet_names = ['AD', 'Homo', 'CH', 'XL']
    with pd.ExcelWriter(output_xlsx) as writer:
        for df, sheet_name in zip([dfs.AD, dfs.Hm, dfs.CH, dfs.XL], sheet_names):
            df.to_excel(writer, sheet_name=sheet_name, index=False)

df_to_excel(dfs, f"{output_file_path}.xlsx")


In [4]:
#-----   STEP (Final Step). Count variants of filtering process
countsummery_file = str(Path(output_file_path).parent) + '/CountSummary.xlsx'
counter_result = counter(dfs=dfs, output_excel=countsummery_file)
print(counter_result)


                     AD   Homo     CH     XL
All variants      11649  11649  11649  11649
Chrom. Filter     11318  11318  11318    331
GT Fitler           156    156   4194    254
MAF Filter           11     13    731     51
In-House Filter       9     12    460     26
Exclude HLA/MUC       9     12    368     26
Exclude Ex.Syno.      9     12    305     19


In [ ]:
#-----   STEP 6. Additional annotations
anno = Anno(df=df, args=args)
df = anno.anno_scores()

#-----   STEP 7. Filtering
maffilter = MafFilter(
    df=df, mode_samples_info=mode_samples_info, config=configs)
df = maffilter.all_filtering()

typefilter = TypeFilter(df=df)
df = typefilter.exclude_hlamuc_and_exonicsyno()

gtfilter = GtFilter(
    df=df, mode_samples_info=mode_samples_info)
dfs = gtfilter.genotypeing_filter()

#-----   STEP 8. Output as an Excel 
def df_to_excel(dfs: dataclass, output_xlsx) -> None:
    sheet_names = ['AD', 'Homo', 'CH', 'XL']
    with pd.ExcelWriter(output_xlsx) as writer:
        for df, sheet_name in zip([dfs.AD, dfs.Hm, dfs.CH, dfs.XL], sheet_names):
            df.to_excel(writer, sheet_name=sheet_name, index=False)

df_to_excel(dfs, f"{output_file_path}.xlsx")

############################################
import pickle
with open(f"{output_file_path}.pkl", mode="wb") as f:
    pickle.dump(dfs, f)
############################################

#-----   STEP (Final Step). Count variants of filtering process
countsummery_file = str(Path(output_file_path).parent) + '/CountSummary.xlsx'
counter_result = counter(dfs=dfs, output_excel=countsummery_file)
print(counter_result)



In [6]:
import pickle
pkl = '/work/Github/TestData/trio/29881/Sample_29881-trio_results/Sample_29881-trio.tsv.pkl'
with open(pkl, mode='rb') as f:
    dfs = pickle.load(f)

print(type(dfs))

<class 'libs.filter.gtfilter.ModelDataFrame'>


In [10]:
hgmd_pkl = '/work/resources/HGMD/hgmd_info_2023.3'
hgmd = pd.read_pickle(hgmd_pkl)

,gene,altsymbol,refseq,expected_inheritance,hgncID,omimid,DFP,DM,DM?,DP,FP,R
0,RBFOX1,2BP1|A2BP1|FOX-1|FOX1|HRNBP1,NM_145891.3,AD,18222,605104,0.0,15.0,47.0,4.0,0.0,0.0
1,ABCA3,ABC-C|ABC3|EST111653|LBM180|SMDP3,NM_001089.3,AR,33,601615,0.0,230.0,152.0,1.0,2.0,0.0
2,AKAP13,AKAP-13|AKAP-Lbc|ARHGEF13|BRX|c-lbc|HA-3|Ht31|...,NM_007200.5,UNK,371,604686,0.0,0.0,11.0,1.0,5.0,0.0
3,GSS,GSHS|HEL-S-64p|HEL-S-88n,NM_000178.4,AR,4624,601002,0.0,40.0,5.0,0.0,0.0,0.0
4,BRF1,BRF|BRF-1|CFDS|GTF3B|hBRF|HEL-S-76p|TAF3B2|TAF...,NM_001519.4,ADAR,11551,604902,0.0,19.0,15.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
17668,TNFRSF21,BM-018|CD358|DR6,NM_014452.5,UNK,13469,605732,0.0,4.0,7.0,0.0,0.0,0.0
17669,C12orf4,MRT66,NM_020374.4,AR,1184,616082,0.0,7.0,4.0,0.0,0.0,0.0
17670,DMBX1,Atx|MBX|OTX3|PAXB,NM_147192.3,UNK,19026,607410,0.0,1.0,5.0,0.0,0.0,0.0
17671,WDR93,C1d-87|CFAP297|FAP297,NM_020212.2,UNK,26924,None,0.0,1.0,1.0,0.0,0.0,0.0


In [ ]:
genocode_file_hg19 = '/resources/GENCODE_Basic_Annotations/gencode.v44lift37.basic.annotation.gtf.gz'
genocode_db_hg19 = '/resources/GENCODE_Basic_Annotations/gencode.v44lift37.basic.annotation.db.bak'

db = gffutils.FeatureDB(genocode_db_hg19)

In [ ]:
import re

@dataclass
class GencodeInfo:
    gene_name: str
    hgnc: str
    ensg: str
    enst: str
    ensg_full: str
    enst_full: str
    strand: str

def __anno_gencode_info(row) -> str:
    query_region: str = f"chr{row['CHROM']}:{row['POS']}-{row['POS']}"
    # query_region: str = f"chr17:1132706-1132706"
    fetched_data = db.region(region=query_region, 
                            featuretype='gene',
                            completely_within=False)
    
    result = []
    while 1:
        try:
            data = next(fetched_data)
        except StopIteration:
            break
        else:
            strand = data.strand
            gene_name = data.attributes['gene_name'][0]
            
            try:
                ensg_full = data.attributes['gene_id'][0]
            except KeyError:
                ensg = '.'
            else:
                ensg = re.match(r'ENSG\d+', ensg_full).group()
    
    # print(result)
    return result

In [ ]:
import re

@dataclass
class GencodeInfo:
    gene_name: str
    hgnc: str
    ensg: str
    enst: str
    ensg_full: str
    enst_full: str
    strand: str

def __anno_gencode_info(row) -> str:
    query_region: str = f"chr{row['CHROM']}:{row['POS']}-{row['POS']}"
    # query_region: str = f"chr17:1132706-1132706"
    fetched_data = db.region(region=query_region, 
                            featuretype='gene',
                            completely_within=False)
    
    result = []
    while 1:
        try:
            data = next(fetched_data)
        except StopIteration:
            break
        else:
            strand = data.strand
            gene_name = data.attributes['gene_name'][0]

            try:
                hgnc_info = data.attributes['hgnc_id'][0]
            except KeyError:
                hgnc = '.'
            else:
                hgnc = re.search(r'\d+', hgnc_info).group()
            
            try:
                ensg_full = data.attributes['gene_id'][0]
            except KeyError:
                ensg = '.'
            else:
                ensg = re.match(r'ENSG\d+', ensg_full).group()
            
            try:
                enst_full = data.attributes['transcript_id'][0]
            except KeyError:
                enst = '.'
            else:
                enst = re.match(r'ENST\d+', enst_full).group()

            genecode_info = GencodeInfo(
                gene_name, hgnc, ensg, enst, ensg_full, enst_full, strand)
            result.append(genecode_info)
    
    # print(result)
    return result
        


In [ ]:
query_region: str = f"chr17:1132706-1132706"
fetched_data = db.region(region=query_region, 
                            featuretype='transcript',
                            completely_within=False)
    
result = []
while 1:
    try:
        data = next(fetched_data)
    except StopIteration:
        break
    else:
        strand = data.strand
        gene_name = data.attributes['gene_name'][0]

        try:
            hgnc_info = data.attributes['hgnc_id'][0]
        except KeyError:
            hgnc = '.'
        else:
            hgnc = re.search(r'\d+', hgnc_info).group()
        
        try:
            ensg_full = data.attributes['gene_id'][0]
        except KeyError:
            ensg = '.'
        else:
            ensg = re.match(r'ENSG\d+', ensg_full).group()
        
        try:
            enst_full = data.attributes['transcript_id'][0]
        except KeyError:
            enst = '.'
        else:
            enst = re.match(r'ENST\d+', enst_full).group()

        genecode_info = GencodeInfo(
            gene_name, hgnc, ensg, enst, ensg_full, enst_fuœll, strand)
        result.append(genecode_info)

In [ ]:
query_region: str = f"chr17:1132706-1132706"
fetched_gene = db.region(region=query_region, 
                         featuretype=['gene', 'transcript'],
                         completely_within=False)


In [ ]:
for data in fetched_gene:
    print(data)

chr17	HAVANA	gene	906759	1133032	.	-	.	gene_id "ENSG00000159842.16_14"; gene_type "protein_coding"; gene_name "ABR"; level "1"; hgnc_id "HGNC:81"; tag "ncRNA_host"; havana_gene "OTTHUMG00000090313.18_14"; remap_status "full_contig"; remap_num_mappings "1"; remap_target_status "overlap";


In [ ]:
gtf = db.children('', featuretype='transcript', order_by='start')

In [ ]:
for g in gtf:
    print(g)
    print(g.attributes['tag'])

In [ ]:

df['gencode'] = df.progress_apply(__anno_gencode_info, axis=1)

100%|██████████| 6443/6443 [01:25<00:00, 75.63it/s] 


In [ ]:
for row in df.iterrows():
    print(row[0], row[1]['Gene.refGene'], row[1]['gencode'])

In [ ]:
import re

query_region: str = f'chr1: 69610-69610'
rows = db.region(region=query_region, featuretype='transcript')

for row in rows:
    print(row)

    enst_full = row.attributes['transcript_id'][0]
    ensg_full = row.attributes['gene_id'][0]
    hgnc_info = row.attributes['hgnc_id'][0]

    try:
        enst = re.match(r'ENST\d+', enst_full).group()
    except AttributeError:
        enst = '.'
    try:
        ensg = re.match(r'ENSG\d+', ensg_full).group()
    except AttributeError:
        ensg = '.'
    try:
        hgnc = re.search(r'\d+', hgnc_info).group()
    except AttributeError:
        hgnc = '.'

    print(f"Strand: {row.strand}")
    print(f"GeneName : {row.attributes['gene_name'][0]}")
    print(f"HGNC_ID  : {hgnc}")
    print(f"ENSG_Full: {ensg_full}")
    print(f"ENSG     : {ensg}")
    print(f"ENST_Full: {enst_full}")
    print(f"ENST     : {enst}")

chr1	HAVANA	transcript	65419	71585	.	+	.	gene_id "ENSG00000186092.7_9"; transcript_id "ENST00000641515.2_5"; gene_type "protein_coding"; gene_name "OR4F5"; transcript_type "protein_coding"; transcript_name "OR4F5-201"; level "2"; protein_id "ENSP00000493376.2"; hgnc_id "HGNC:14825"; tag "RNA_Seq_supported_partial"; tag "basic"; tag "Ensembl_canonical"; tag "MANE_Select"; tag "appris_principal_1"; tag "CCDS"; ccdsid "CCDS30547.2"; havana_gene "OTTHUMG00000001094.4_9"; havana_transcript "OTTHUMT00000003223.4_5"; remap_num_mappings "1"; remap_status "full_contig"; remap_target_status "new";
Strand: +
ENSG_Full: ENSG00000186092.7_9
ENSG     : ENSG00000186092
ENST_Full: ENST00000641515.2_5
ENST     : ENST00000641515
GeneName : OR4F5
HGNC_ID  : 14825


In [ ]:
#-----   STEP 5. Pre-processing
preprocessing = PreProcessExomeSummary(
    df=df, mode_samples_info=mode_samples_info)
df = preprocessing.all_pre_processing()

#-----   STEP 6. Additional annotations
anno = Anno(df=df, args=args)
df = anno.anno_scores()

#-----   STEP 7. Filtering
maffilter = MafFilter(
    df=df, mode_samples_info=mode_samples_info, config=config)
df = maffilter.all_filtering()

typefilter = TypeFilter(df=df)
df = typefilter.exclude_hlamuc_and_exonicsyno()

gtfilter = GtFilter(
    df=df, mode_samples_info=mode_samples_info)
dfs = gtfilter.genotypeing_filter()

#-----   STEP 9. Output as an Excel 
def df_to_excel(dfs: dataclass, output_xlsx) -> None:
    sheet_names = ['AD', 'Homo', 'CH', 'XL']
    with pd.ExcelWriter(output_xlsx) as writer:
        for df, sheet_name in zip([dfs.AD, dfs.Hm, dfs.CH, dfs.XL], sheet_names):
            df.to_excel(writer, sheet_name=sheet_name, index=False)

output_xlsx = f"{output_file_path}.xlsx"
output_pickle = f"{output_file_path}.pkl"
df_to_excel(dfs, output_xlsx)
dfs.to_pickle(output_pickle)

#-----   STEP 8. Count variants of filtering process
countsummery_file = str(Path(output_file_path).parent) + '/CountSummary.xlsx'
counter_result = counter(dfs=dfs, output_excel=countsummery_file)
print(counter_result)


In [ ]:
dfs.AD.columns

Index(['InHouse_absent_FILTER', 'InHouse_1%_FILTER', 'MAF_0.1%_FILTER',
       'MAF_1%_FILTER', 'HLAMUC_FILTER', 'ExonicSyno_FILTER', 'GT_FILTER'],
      dtype='object')

In [ ]:
print(output_file_path)

/work/Github/TestData/trio/29881/Sample_29881-trio_results/Sample_29881-trio.tsv


In [ ]:
#-----   STEP 9. Output as an Excel 
def df_to_excel(dfs: dataclass, output_xlsx) -> None:
    sheet_names = ['AD', 'Homo', 'CH', 'XL']
    with pd.ExcelWriter(output_xlsx) as writer:
        for df, sheet_name in zip([dfs.AD, dfs.Hm, dfs.CH, dfs.XL], sheet_names):
            df.to_excel(writer, sheet_name=sheet_name, index=False)

output_xlsx = f"{output_file_path}.xlsx"
df_to_excel(dfs, output_xlsx)

In [ ]:
hgmd_resource: str = [
    str(x) for x in self.resources_dir.glob('HGMD/HGMD_gene_based.tsv.gz')][0]
        return pd.read_csv(hgmd_resource, sep='\t')

'Sample_32741-proband_countsummary.txt'

In [ ]:
couter_result = counter(dfs=dfs, output_excel=output_file_path)


In [ ]:
df = counter(dfs=dfs, output_excel='./test.xlsx')

In [ ]:
df.to_pickle('./post7.pkl')

In [ ]:
df = pd.read_pickle('./post7.pkl')

In [ ]:
df = df.head(100)

In [ ]:
#-----   STEP 8. Output as an Excel 
output_xlsx = './head100.xlsx'
def df_to_excel(dfs: dataclass, output_xlsx) -> None:
    sheet_names = ['AD', 'Homo', 'CH', 'XL']
    with pd.ExcelWriter(output_xlsx) as writer:
        for df, sheet_name in zip([dfs.AD, dfs.Hm, dfs.CH, dfs.XL], sheet_names):
            df.to_excel(writer, sheet_name=sheet_name, index=False)

df_to_excel(dfs, 'output_xlsx')

In [ ]:
#----   STEP 9. Insert hyperlinks
# 0. 最初にエクセルを読み込む
# 1. メモ用の列を作る (1-2列目)
# 2. リンク挿入用の列を作る (3-7列目)
# 3. リンクの挿入

In [ ]:
from libs.excelibs.excel_format import ExcelFormat
from libs.excelibs.hyperlinks import HyperLinks

excelformat = ExcelFormat('./test1.xlsx')

Formatng ......


In [ ]:
excelformat.insert_comment_cols()

In [ ]:
excelformat.insert_hyperlink_cols()

In [ ]:
excelformat.workbook.save('./test2.xlsx')

##### CREATE dbs

In [ ]:
## Create db for gffutils
import gffutils
import gffutils.pybedtools_integration


genocode_file_hg19 = '/resources/GENCODE_Basic_Annotations/gencode.v44lift37.basic.annotation.gtf.gz'
genocode_db_hg19 = '/resources/GENCODE_Basic_Annotations/gencode.v44lift37.basic.annotation.db'


In [ ]:
db = gffutils.create_db(data=genocode_file_hg19, dbfn=genocode_db_hg19, 
                        disable_infer_genes=True,
                        disable_infer_transcripts=True,
                        keep_order=True, 
                        force=True)

In [ ]:
genocode_db_intron_hg19 = '/resources/GENCODE_Basic_Annotations/gencode.v44lift37.basic.annotation.intron.db'

db = gffutils.FeatureDB(genocode_db_hg19)
introns = db.create_introns(exon_featuretype='exon', 
                            new_featuretype='intron', 
                            merge_attributes=True, 
                            numeric_sort=True)
pybed = gffutils.pybedtools_integration.to_bedtool(introns)
pybed.saveas(genocode_db_intron_hg19)

<BedTool(/resources/GENCODE_Basic_Annotations/gencode.v44lift37.basic.annotation.intron.db)>

In [ ]:
def generate_intoron_gtf(db: gffutils.FeatureDB, output: str) -> None:
    introns = db.create_introns(exon_featuretype='exon', 
                                new_featuretype='intron', 
                                merge_attributes=True, 
                                numeric_sort=True)
    pybed = gffutils.pybedtools_integration.to_bedtool(introns)
    pybed.saveas(output)
    
    return